In [ ]:
# PlantVillage Training Notebook
# - Downloads dataset via kagglehub
# - Trains a Keras CNN
# - Exports TFJS model for frontend

import os
import pathlib

print("Working dir:", os.getcwd())

Working dir: c:\Users\np792\Downloads\crop-disease-detector\crop-disease-detector\ml


In [2]:
# Install deps (local env)
%pip -q install kagglehub tensorflow tensorflowjs matplotlib scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Download PlantVillage
import kagglehub
path = kagglehub.dataset_download("emmarex/plantdisease")
print("Path to dataset files:", path)

c:\Users\np792\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\np792\.cache\kagglehub\datasets\emmarex\plantdisease\versions\1


In [3]:
# Prepare data generators
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataroot = pathlib.Path(path)
img_size = (224, 224)
batch_size = 32

train_dir = dataroot / 'PlantVillage'
if not train_dir.exists():
    # Some archives expand differently; adjust here if needed
    # Fallback to root
    train_dir = dataroot

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='sparse'
)
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='sparse'
)

class_names = list(train_gen.class_indices.keys())
print('Classes:', len(class_names))

Found 33027 images belonging to 16 classes.
Found 8249 images belonging to 16 classes.
Classes: 16


In [4]:
# Build a simple CNN (or swap with Transfer Learning)
from tensorflow.keras import layers, models

base = tf.keras.applications.MobileNetV2(
    include_top=False, input_shape=(224,224,3), weights='imagenet')
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │        20,496 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,278,480 (8.69 MB)

 Trainable params: 20,496 (80.06 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
# Train
steps = train_gen.samples // batch_size
val_steps = val_gen.samples // batch_size

history = model.fit(
    train_gen,
    steps_per_epoch=steps,
    validation_data=val_gen,
    validation_steps=val_steps,
    epochs=5
)

Epoch 1/5
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 804s 775ms/step - accuracy: 0.5288 - loss: 1.1761 - val_accuracy: 0.2363 - val_loss: 2.7232
Epoch 2/5
   1/1032 ━━━━━━━━━━━━━━━━━━━━ 8:21 487ms/step - accuracy: 0.4062 - loss: 1.1029

c:\Users\np792\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1032/1032 ━━━━━━━━━━━━━━━━━━━━ 160s 155ms/step - accuracy: 0.4062 - loss: 1.1029 - val_accuracy: 0.2265 - val_loss: 2.7042
Epoch 3/5
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 699s 677ms/step - accuracy: 0.5471 - loss: 1.0076 - val_accuracy: 0.1818 - val_loss: 3.0320
Epoch 4/5
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 129s 125ms/step - accuracy: 0.4688 - loss: 1.2264 - val_accuracy: 0.1773 - val_loss: 3.0185
Epoch 5/5
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 687s 666ms/step - accuracy: 0.5463 - loss: 0.9851 - val_accuracy: 0.2419 - val_loss: 3.3721


In [7]:
import pathlib
import json

out_dir = pathlib.Path('export_tfjs')
out_dir.mkdir(exist_ok=True)

# Save labels
labels_json = out_dir / 'labels.json'
with open(labels_json, 'w') as f:
    json.dump(class_names, f)

# Save as Keras format and convert
model.save(out_dir / 'model.h5')

print('Saved model.h5')
print('\nNow run:')
print(f'tensorflowjs_converter --input_format=keras {out_dir.as_posix()}/model.h5 {out_dir.as_posix()}')

Saved model.h5

Now run:
tensorflowjs_converter --input_format=keras export_tfjs/model.h5 export_tfjs


In [8]:
!pip install --upgrade tensorflow-hub==0.15.0

  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.12.0
    Uninstalling tensorflow-hub-0.12.0:
      Successfully uninstalled tensorflow-hub-0.12.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflowjs 3.18.0 requires tensorflow-hub<0.13,>=0.7.0, but you have tensorflow-hub 0.15.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pathlib
import json
import subprocess

out_dir = pathlib.Path('export_tfjs')
out_dir.mkdir(exist_ok=True)

# Save labels
with open(out_dir / 'labels.json', 'w') as f:
    json.dump(class_names, f)

# Save model
model_path = out_dir / 'model.keras'
model.save(model_path.as_posix())

# Convert using subprocess
print('Converting model...')
result = subprocess.run([
    'tensorflowjs_converter',
    '--input_format=keras',
    model_path.as_posix(),
    out_dir.as_posix()
], capture_output=True, text=True)

if result.returncode == 0:
    print('✓ Conversion successful!')
    print(f'✓ Files saved to: {out_dir.resolve()}')
else:
    print('Error:', result.stderr)

Converting model...
Error: 2025-11-04 04:57:17.822236: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-04 04:57:19.815287: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
C:\Users\np792\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflowjs\read_weights.py:28: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.uint8, np.uint16, np.object, np.bool]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  Fi